## Reading Data

In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cd ASD && ls | wc -l
asd_files = os.listdir('/content/drive/MyDrive/Speech/FinalData/ASD/')

/bin/bash: line 0: cd: ASD: No such file or directory


In [ ]:
! cd TD && ls | wc -l
td_files = os.listdir('/content/drive/MyDrive/Speech/FinalData/TD/')

/bin/bash: line 0: cd: TD: No such file or directory


In [ ]:
import re
from tqdm import tqdm

In [ ]:
import numpy

In [ ]:
import nltk
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sentence = 'i am preprocessing the text'
words = nltk.word_tokenize(sentence)

In [ ]:
cd /content/drive/MyDrive/Speech/FinalData/

/content/drive/MyDrive/Speech/FinalData


In [ ]:
df = pd.read_csv('childesData.csv', index_col = 0)

In [ ]:
df.head(10)

,ID,ChildText,Label,ChildTheraText
0,060622.cha,these are Brett's . | drop them off . | do wi...,0,what do you do ? | mix the stuff in . $ right...
1,030900.cha,want lunch . | want some lunch . | have some ...,0,roger want some lunch ? | want lunch . $ let'...
2,080506.cha,"hi , Helen . | hi Tina . | xxx . | hi Tina . ...",0,"okay ? | hi , Helen . $ did you say hi to Tin..."
3,080808.cha,nah . | don't do that . | no . | I don't care...,0,"yeah , come on . | don't do that . $ come on ..."
4,1025.cha,oh ‡ I found a jelly+bean . | I wanna eat it ...,1,you found a jelly+bean . | I wanna eat it . $...
5,040715.cha,"come . | ready , go . | &mm . | &mm . | doggy...",0,what do you say ? | come . $ open it . | dogg...
6,111.cha,&-um make a xxx and dump . | make dump . | bi...,1,a what ? | make dump . $ is that it ? | yeah ...
7,030806.cha,want water . | water . | sit down . | sit rig...,0,Stuart . | want water . $ you want water ? | ...
8,120.cha,beep@o beep@o . | &ah xxx . | &sh . | one . |...,0,"hm| &ah xxx . $ wow, yeah . | &sh . $ yeah, c..."
9,billy2.cha,ni(ght)_night . | ni(ght)_night . | are these...,1,where is the baby ? | ni(ght)_night . $ what ...


##Metaphor Detection

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("lwachowiak/Metaphor-Detection-XLMR")
tokenizer_h = AutoTokenizer.from_pretrained("ganeshkharad/gk-hinglish-sentiment")

model = AutoModelForTokenClassification.from_pretrained("lwachowiak/Metaphor-Detection-XLMR")

pipeline_metaphors=pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
pipeline_metaphors_h=pipeline("ner", model=model, tokenizer=tokenizer_h, aggregation_strategy="simple")

In [ ]:
def metaphors(text,num_count):
  dic=pipeline_metaphors(text)
  total=0
  for i in dic:
    if (i['entity_group']=='LABEL_1'):
      total+=1
  return total/num_count

In [ ]:
def metaphors_h(text,num_count):
  dic=pipeline_metaphors_h(text)
  total=0
  for i in dic:
    if (i['entity_group']=='LABEL_1'):
      total+=1
  return total/num_count

## No. of words

In [ ]:
def num(words):
    count=0
    list= []
    for word in words:
      if word in list:
        continue
      else:
        count+=1
        list.append(word)
    return len(list)

## MLUM

In [ ]:
!pip install polyglot

In [ ]:
!pip install pyicu
!pip3 install pycld2
!pip3 install morfessor

Reference: https://polyglot.readthedocs.io/en/latest/MorphologicalAnalysis.html

In [ ]:
from polyglot.downloader import downloader
print(downloader.supported_languages_table("morph2"))

In [ ]:
%%bash
polyglot download morph2.en morph2.ar morph2.hi

[polyglot_data] Downloading package morph2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package morph2.ar to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package morph2.hi to
[polyglot_data]     /root/polyglot_data...


In [ ]:
from polyglot.text import Text, Word
from collections import Counter

In [ ]:
def mlum(words):
    num = 0
    den = 0
    for w in words:
        w = Word(w, language="en")
        for i in w.morphemes:
            num += len(i)
            den += 1
    ml = num/den
    return ml

In [ ]:
def mlum_h(words):
    num = 0
    den = 0
    for w in words:
        w = Word(w, language="hi")
        for i in w.morphemes:
            num += len(i)
            den += 1
    ml = num/den
    return ml

## NDWR

In [ ]:
import re
import os
try:
    import nltk
except:
    print("First install NLTK using pip install nltk command")
    exit()
try:
    import gensim
    from gensim import corpora, models, similarities
except:
    print("First install Gensim using pip install nltk command")
    exit()


class Stemmer:
    w2vModel = None
    sensitivity = 10

    # constructor
    def __init__(self, modelLocation= "w2vModel", w2vModel = None):
        try:
            self.w2vModel = gensim.models.Word2Vec.load(modelLocation)
        except:
            print("Could not locate the w2vModel file in the directory : "+(modelLocation))
            print("Try to load the w2vModel and try again")



    #  ----------- Stemming functions -----------


    # takes a word and removes the repeated occurance of characters in that word
    # outputs word without repeat consecutive occurance of the word
    def RepetitionStemmer(self, word):
        # find repeted occurence of letters in a word
        # remove the occurence
        i=0
        newWord = ''
        while(i <len(word)):
            c = word[i]
            newWord+=c
            while(i<len(word) and word[i] == c):
                i=i+1

        return newWord

    # takes a word2vec model, word and nWords(to run most similar on - higher the better but slower)
    # output the list of words similar to that word ( including that word passed through repetition stemmer)
    def WordEmbeddingStemmer(self, w2vModel, word, nWords = 10):

        try:
            similarWordsList =[w2vModel.wv.most_similar(word, topn = nWords )[i][0] for i in range(10)]
        except:
            return self.RepetitionStemmer(word)

        word = self.RepetitionStemmer(word)

        outputList = []
        for similarWord in similarWordsList:
            stemmSimilarWord = self.RepetitionStemmer(similarWord)
            w0 = word
            w1 = word[:-1]
            sw0 = stemmSimilarWord
            sw1 = stemmSimilarWord[:-1]

            if (sw0 in w0) or( w0 in sw0) or (sw1 in w0) or (w1 in sw0):
                if(len(stemmSimilarWord)<len(word)):
                    outputList.append(stemmSimilarWord)
                else:
                    outputList.append(word)
        if len(outputList) == 0:
            outputList.append(word)
        return outputList[0]

    # stemmers
    def stemWord(self, word):
        return self.WordEmbeddingStemmer(self.w2vModel, word)

    def stemListOfWords(self, listOfWords):
        return [self.WordEmbeddingStemmer(self.w2vModel, word) for word in listOfWords]

    def stem2dListOfWords(self, listOfWords2d):
        output = []
        for sentenceOfWords in listOfWords2d:
            output.append([self.WordEmbeddingStemmer(self.w2vModel, word) for word in sentenceOfWords])
        return output

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
myStemmer = Stemmer()
porter = PorterStemmer()

In [ ]:
def ndwr(words):
    ndwr = Counter()
    for w in words:
        ndwr[porter.stem(w)] = 1
    return len(ndwr.keys())

In [ ]:
def ndwr_h(words):
    ndwr = Counter()
    for w in words:
        ndwr[myStemmer.stemWord(w)] = 1
    return len(ndwr.keys())

## Um prop

In [ ]:
def um_prop(words):
    num = 0
    den = 0
    for w in words:
        f1 = 0
        f2 = 0
        for l in w:
            if l!= 'u' or l!= 'h':
                f1 = 1
            if l!= 'u' or l!='m':
                f2 = 1
        if(f1 == 0):
            den += 1
        if(f2 == 0):
            num += 1
            den += 1
    try:
        um_prop = (num/den)
    except:
        um_prop = 0
    return um_prop

## Unintell Prop

In [ ]:
def unintell(cunits):
    ct = 0
    for c in cunits:
        if('xxx' in c or 'yyy' in c):
            ct += 1
    return ct/len(cunits)

In [ ]:
def unintell_h(cunits):
    ct = 0
    for c in cunits:
      if ((c.count('X')>0) or (c.count('x')>0)):
        ct+=1
    return ct/len(cunits)

## Average length of C-Unit

In [ ]:
def avg_cunit(cunits):
    l = 0
    for i in cunits:
        l += len(i)
    return l/len(cunits)

## Number of C-Unit

In [ ]:
def num_cunit(cunits):
    return len(cunits)

## Repetition Prop

In [ ]:
def repetitionProp(s):
    totCnt = 0
    for i in range(1, len(s)):
        bigrams = Counter()
        ther = nltk.word_tokenize(s[i-1])
        chil = nltk.word_tokenize(s[i])
        for i in range(1, len(ther)):
            bigrams[ther[i-1] + ' ' + ther[i]] = 1
        for i in range(1, len(chil)):
            if(bigrams[chil[i-1] + ' ' + chil[i]] == 1):
                totCnt += 1
    return totCnt

In [ ]:
!pip install inltk

In [ ]:
from inltk.inltk import setup
from inltk.inltk import tokenize

def repetitionProp_h(s):
    totCnt = 0
    for i in range(1, len(s)):
        bigrams = Counter()
        ther = tokenize(s[i-1], 'hi-en')
        chil = tokenize(s[i],'hi-en')
        for i in range(1, len(ther)):
            bigrams[ther[i-1] + ' ' + ther[i]] = 1
        for i in range(1, len(chil)):
            if(bigrams[chil[i-1] + ' ' + chil[i]] == 1):
                totCnt += 1
    return totCnt

## Echolalia

In [ ]:
def echolalia(c):
    totCnt = 0
    for i in range(len(c)):
        bigrams = Counter()
        ther = nltk.word_tokenize(c[i][0])
        chil = nltk.word_tokenize(c[i][1])
        for i in range(1, len(ther)):
            bigrams[ther[i-1] + ' ' + ther[i]] = 1
        for i in range(1, len(chil)):
            if(bigrams[chil[i-1] + ' ' + chil[i]] == 1):
                totCnt += 1
    return totCnt

In [ ]:
def echolalia_h(c):
    totCnt = 0
    for i in range(len(c)):
        bigrams = Counter()
        ther = tokenize(c[i][0], 'hi-en')
        chil = tokenize(c[i][1], 'hi-en')
        for i in range(1, len(ther)):
            bigrams[ther[i-1] + ' ' + ther[i]] = 1
        for i in range(1, len(chil)):
            if(bigrams[chil[i-1] + ' ' + chil[i]] == 1):
                totCnt += 1
    return totCnt

## Unexpected Words

In [ ]:
hinglish="a aadi aaj aap aapne aata aati aaya aaye ab abbe abbey abe abhi able about above accha according accordingly acha achcha across actually after afterwards again against agar ain aint ain't aisa aise aisi alag all allow allows almost alone along already also although always am among amongst an and andar another any anybody anyhow anyone anything anyway anyways anywhere ap apan apart apna apnaa apne apni appear are aren arent aren't around arre as aside ask asking at aur avum aya aye baad baar bad bahut bana banae banai banao banaya banaye banayi banda bande bandi bane bani bas bata batao bc be became because become becomes becoming been before beforehand behind being below beside besides best better between beyond bhai bheetar bhi bhitar bht bilkul bohot bol bola bole boli bolo bolta bolte bolti both brief bro btw but by came can cannot cant can't cause causes certain certainly chahiye chaiye chal chalega chhaiye clearly c'mon com come comes could couldn couldnt couldn't d de dede dega degi dekh dekha dekhe dekhi dekho denge dhang di did didn didnt didn't dijiye diya diyaa diye diyo do does doesn doesnt doesn't doing done dono dont don't doosra doosre down downwards dude dunga dungi during dusra dusre dusri dvaara dvara dwaara dwara each edu eg eight either ek else elsewhere enough etc even ever every everybody everyone everything everywhere ex exactly example except far few fifth fir first five followed following follows for forth four from further furthermore gaya gaye gayi get gets getting ghar given gives go goes going gone good got gotten greetings haan had hadd hadn hadnt hadn't hai hain hamara hamare hamari hamne han happens har hardly has hasn hasnt hasn't have haven havent haven't having he hello help hence her here hereafter hereby herein here's hereupon hers herself he's hi him himself his hither hm hmm ho hoga hoge hogi hona honaa hone honge hongi honi hopefully hota hotaa hote hoti how howbeit however hoyenge hoyengi hu hua hue huh hui hum humein humne hun huye huyi i i'd idk ie if i'll i'm imo in inasmuch inc inhe inhi inho inka inkaa inke inki inn inner inse insofar into inward is ise isi iska iskaa iske iski isme isn isne isnt isn't iss isse issi isski it it'd it'll itna itne itni itno its it's itself ityaadi ityadi i've ja jaa jab jabh jaha jahaan jahan jaisa jaise jaisi jata jayega jidhar jin jinhe jinhi jinho jinhone jinka jinke jinki jinn jis jise jiska jiske jiski jisme jiss jisse jitna jitne jitni jo just jyaada jyada k ka kaafi kab kabhi kafi kaha kahaa kahaan kahan kahi kahin kahte kaisa kaise kaisi kal kam kar kara kare karega karegi karen karenge kari karke karna karne karni karo karta karte karti karu karun karunga karungi kaun kaunsa kayi kch ke keep keeps keh kehte kept khud ki kin kine kinhe kinho kinka kinke kinki kinko kinn kino kis kise kisi kiska kiske kiski kisko kisliye kisne kitna kitne kitni kitno kiya kiye know known knows ko koi kon konsa koyi krna krne kuch kuchch kuchh kul kull kya kyaa kyu kyuki kyun kyunki lagta lagte lagti last lately later le least lekar lekin less lest let let's li like liked likely little liya liye ll lo log logon lol look looking looks ltd lunga m maan maana maane maani maano magar mai main maine mainly mana mane mani mano many mat may maybe me mean meanwhile mein mera mere merely meri might mightn mightnt mightn't mil mjhe more moreover most mostly much mujhe must mustn mustnt mustn't my myself na naa naah nahi nahin nai name namely nd ne near nearly necessary neeche need needn neednt needn't needs neither never nevertheless new next nhi nine no nobody non none noone nope nor normally not nothing novel now nowhere o obviously of off often oh ok okay old on once one ones only onto or other others otherwise ought our ours ourselves out outside over overall own par pata pe pehla pehle pehli people per perhaps phla phle phli placed please plus poora poori provides pura puri q que quite raha rahaa rahe rahi rakh rakha rakhe rakhen rakhi rakho rather re really reasonably regarding regardless regards rehte rha rhaa rhe rhi ri right s sa saara saare saath sab sabhi sabse sahi said sakta saktaa sakte sakti same sang sara sath saw say saying says se second secondly see seeing seem seemed seeming seems seen self selves sensible sent serious seriously seven several shall shan shant shan't she she's should shouldn shouldnt shouldn't should've si since six so soch some somebody somehow someone something sometime sometimes somewhat somewhere soon still sub such sup sure t tab tabh tak take taken tarah teen teeno teesra teesre teesri tell tends tera tere teri th tha than thank thanks thanx that that'll thats that's the theek their theirs them themselves then thence there thereafter thereby therefore therein theres there's thereupon these they they'd they'll they're they've thi thik thing think thinking third this tho thoda thodi thorough thoroughly those though thought three through throughout thru thus tjhe to together toh too took toward towards tried tries true truly try trying tu tujhe tum tumhara tumhare tumhari tune twice two um umm un under unhe unhi unho unhone unka unkaa unke unki unko unless unlikely unn unse until unto up upar upon us use used useful uses usi using uska uske usne uss usse ussi usually vaala vaale vaali vahaan vahan vahi vahin vaisa vaise vaisi vala vale vali various ve very via viz vo waala waale waali wagaira wagairah wagerah waha wahaan wahan wahi wahin waisa waise waisi wala wale wali want wants was wasn wasnt wasn't way we we'd well we'll went were we're weren werent weren't we've what whatever what's when whence whenever where whereafter whereas whereby wherein where's whereupon wherever whether which while who whoever whole whom who's whose why will willing with within without wo woh wohi won wont won't would wouldn wouldnt wouldn't y ya yadi yah yaha yahaan yahan yahi yahin ye yeah yeh yehi yes yet you you'd you'll your you're yours yourself yourselves you've yup"
hinglish_stop_words=hinglish.split(" ")
print(hinglish_stop_words)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words={'english'})
def unexpected_words(c):
    X = vectorizer.fit_transform(c)
    return (X > 0.9).sum()

In [ ]:
vectorizer_h = TfidfVectorizer(stop_words=hinglish_stop_words)
def unexpected_words_h(c):
    X = vectorizer_h.fit_transform(c)
    return (X > 0.9).sum()

## Discourse Coherence

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('aditeyabaral/sentencetransformer-bert-hinglish-small')

In [ ]:
def get_bert_embeddings(sentences):
    return sbert_model.encode(sentences)
def get_bert_embeddings_h(sentences):
  return model.encode(sentences)

In [ ]:
from scipy import spatial
def cosine_sim(emb1, emb2):
    return 1 - spatial.distance.cosine(emb1, emb2)

In [ ]:
def discourse_h(s):
    embeddings = get_bert_embeddings_h(s)
    ans = 0
    for i in range(1, len(embeddings)):
        ans += cosine_sim(embeddings[i-1], embeddings[i])
    return ans/len(s)

def discourse(s):
    embeddings = get_bert_embeddings(s)
    ans = 0
    for i in range(1, len(embeddings)):
        ans += cosine_sim(embeddings[i-1], embeddings[i])
    return ans/len(s)

In [ ]:
def discourse_2(c2):
    c = []
    s = []
    for i in range(len(c2)):
        c.append(c2[i][1])
        s.append(c2[i][0])
    embeddings = get_bert_embeddings(c)
    embeddings_2 = get_bert_embeddings(s)
    ans = 0
    for i in range(len(embeddings)):
        ans += cosine_sim(embeddings[i], embeddings_2[i])
    return ans/len(s)

def discourse_2_h(c2):
    c = []
    s = []
    for i in range(len(c2)):
        c.append(c2[i][1])
        s.append(c2[i][0])
    embeddings = get_bert_embeddings_h(c)
    embeddings_2 = get_bert_embeddings_h(s)
    ans = 0
    for i in range(len(embeddings)):
        ans += cosine_sim(embeddings[i], embeddings_2[i])
    return ans/len(s)

##Syntactic Density


In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

from scipy import spatial
def cosine_sim(emb1, emb2):
    return 1 - spatial.distance.cosine(emb1, emb2)

##POS Tags

In [ ]:
pip install stanfordnlp

In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en')


In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')

In [ ]:
def pos_tags(paragraph):
  doc = nlp(paragraph)
  num = 0
  tags = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for sentence in doc.sentences:
    for word in sentence.words:
      num += 1
      tag = word.upos
      if (tag == 'ADJ'):
        tags[0]+=1
        continue
      if (tag == 'ADP'):
        tags[1]+=1
        continue
      if (tag == 'ADV'):
        tags[2]+=1
        continue
      if (tag == 'AUX'):
        tags[3]+=1
        continue
      if (tag == 'CCONJ'):
        tags[4]+=1
        continue
      if (tag == 'DET'):
        tags[5]+=1
        continue
      if (tag == 'INTJ'):
        tags[6]+=1
        continue
      if (tag == 'NOUN'):
        tags[7]+=1
        continue
      if (tag == 'NUM'):
        tags[8]+=1
        continue
      if (tag == 'PART'):
        tags[9]+=1
        continue
      if (tag == 'PRON'):
        tags[10]+=1
        continue
      if (tag == 'PROPN'):
        tags[11]+=1
        continue
      if (tag == 'SCONJ'):
        tags[12]+=1
        continue
      if (tag == 'VERB'):
        tags[13]+=1
        continue
  for i in range(len(tags)):
    tags[i] = tags[i]/num
  return tags

##N-grams

In [ ]:
from nltk.util import ngrams
import re

def n_grams(text):
  sentences = re.split(" .  ? ",text)
  uni=[]
  bi=[]
  tri=[]
  for sentence in sentences:
    unigrams = ngrams(sentence.split(), 1)
    bigrams= ngrams(sentence.split(), 2)
    trigrams= ngrams(sentence.split(), 3)
    uni.extend(unigrams)
    bi.extend(bigrams)
    tri.extend(trigrams)
  uni_sorted=sorted_grams(uni,1)
  bi_sorted=sorted_grams(bi,2)
  tri_sorted=sorted_grams(tri,3)
  return uni_sorted, bi_sorted, tri_sorted

def sorted_grams(ngrams, n=3):
    counts = {}
    for ngram in ngrams:
        counts[ngram] = counts.get(ngram, 0) + 1

    return sorted(((v,k) for k,v in counts.items()), reverse=True)

for i in range(len(df['ChildText'])):
    if(type(df['ChildText'][i]) == float):
        df['ChildText'][i] = ""
    cur = []
    cunits = list(df['ChildText'][i].split('|'))
    tunits = list(df['ChildTheraText'][i].split('$'))
    ftu = []
    for ii in range(len(tunits)):
        tp = list(tunits[ii].split('|'))
        if(len(tp) == 2):
            ftu.append(tp)
    text = df['ChildText'][i].replace('|', '')
    print(text)
    uni_sorted,bi_sorted= n_grams(text)
    print(uni_sorted)
    break

## Extracting Features

In [ ]:
!pip install torch==1.13.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install nest_asyncio

from inltk.inltk import setup
from inltk.inltk import tokenize
import nest_asyncio

In [ ]:
nest_asyncio.apply()
setup("hi-en")

In [ ]:
df_hindi = pd.read_csv('childesData_hinglish.csv')

In [ ]:
from tqdm.auto import tqdm

In [ ]:
features_hindi = []
for i in tqdm(range(len(df_hindi['ChildText']))):
    cur = []
    print(df_hindi['ChildText'][i])
    cunits = list(df_hindi['ChildText'][i].split('.'))
    tunits = list(df_hindi['ChildTheraText'][i].split('$'))

    ftu = []
    print(cunits)
    if (len(cunits)==1):
      continue
    for ii in range(len(tunits)):
        tp = list(tunits[ii].split('|'))
        if(len(tp) == 2):
            ftu.append(tp)
    # CHECK FOR HINGLISH POS TAGGING
    text = df['ChildText'][i]
    cur.append(df['Label'][i])
    words = tokenize(text, 'hi-en')
    try:
        cur.append(mlum_h(words))
    except:
        cur.append(0)
    #id= df["ID"][i][:-4]
    num_words=ndwr_h(words)
    cur.append(num_words)
    print("Ndwr = ",num_words)
    cur.append(um_prop(words))
    print("Um Prop = ",um_prop(words))
    cur.append(avg_cunit(cunits))
    print("Avg C-unit = ",avg_cunit(cunits))
    cur.append(num_cunit(cunits))
    print("Num Cunits = ",num_cunit(cunits))
    cur.append(unintell_h(cunits))
    print("Unintelligible C-units = ",unintell_h(cunits))
    rep = repetitionProp_h(cunits)/len(cunits)
    cur.append(rep)
    print("Repitition Prop = ",rep)
    echo = echolalia(ftu)
    cur.append(echo)
    print("Echolalia = ",echo)
    discourse1 = discourse_h(cunits)
    cur.append(discourse1)
    print("Discourse 1 = ",discourse1)
    discourse2 = discourse_2_h(ftu)
    cur.append(discourse2)
    print("Discourse 2 = ",discourse2)
    cur.append(unexpected_words_h(cunits))
    print("Unexpected Words = ",unexpected_words_h(cunits))
    cur.append(metaphors_h(text,num_words))
    print("Metaphors = ",metaphors_h(text,num_words))
    cur.append(text.count('?'))
    print("Questions = ",text.count('?'))
    features_hindi.append(cur)

In [ ]:
df_hindi_features = pd.DataFrame(features_hindi)
df_hindi_features.to_csv('features_hindi.csv')

In [ ]:
df = pd.DataFrame(features)
df.to_csv('features_f.csv')

In [ ]:
for i in tqdm(range(len(df['ChildText']))):
    if(type(df['ChildText'][i]) == float):
        df['ChildText'][i] = ""
    text = df['ChildText'][i].replace('|', '')
    f = open('Text_CohMetrix/' + df['ID'][i][:-4] + '.txt', 'w')
    f.write(text)
    f.close()

100%|██████████| 165/165 [00:00<00:00, 206.02it/s]


In [ ]:
for i in tqdm(range(len(df['ChildText']))):
    if(type(df['ChildText'][i]) == float):
        df['ChildText'][i] = ""
    text = df['ChildText'][i].replace('|', '')
    f = open('TextFiles/' + str(i) + '.txt', 'w')
    f.write(text)
    f.close()

100%|██████████| 165/165 [00:00<00:00, 203.51it/s]


## Miscellaneous

In [ ]:
def getChildSentences(lines):
    lineCont = False
    text = ""
    for l in lines:
        print(l)
        if l[0] == '@' or l[0] == '%' or l[0] == '*':
            if(lineCont == True):
                lineCont = False
                text += '|'
        if l[:5] == '*CHI:':
            lineCont = True
            text += list(l.split(':'))[1]
        elif lineCont == True:
            text += " " + l
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = re.sub("\[.*?\]","",text)
    text = text.replace('  ', ' ')
    return text

In [ ]:
def getChildThera(lines):
    text = ""
    for i in range(3, len(lines)):
        if(lines[i-3][0] == '*' and lines[i-3][:5] != '*CHI:' and lines[i][:5] == '*CHI:'):
            text += list(lines[i-3].split(':'))[1]
            text += "|"
            text += list(lines[i].split(':'))[1]
            text += "$"
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = re.sub("\[.*?\]","",text)
    text = text.replace('  ', ' ')
    return text

In [ ]:
data = []
for fname in tqdm(asd_files):
    lines = open('ASD/' + fname, 'r').readlines()
    text = getChildSentences(lines)
    data.append([fname, text, 0, getChildThera(lines)])

100%|██████████| 93/93 [00:23<00:00,  3.93it/s]


In [ ]:
for fname in tqdm(td_files):
    lines = open('TD/' + fname, 'r').readlines()
    text = getChildSentences(lines)
    data.append([fname, text, 1, getChildThera(lines)])

In [ ]:
df = pd.DataFrame(data, columns = ['ID', 'ChildText', 'Label', 'ChildTheraText'])

In [ ]:
df = df.sample(frac = 1).reset_index(drop=True)

In [ ]:
df.to_csv('childesData.csv')

##Hinglish Processing

In [ ]:
def getChildSentences(lines):
    lineCont = False
    text = ""
    for l in lines:
        if l[0]=='C':
          l = l.replace('[CS]', '')
          l = l.replace('[FP]', '')
          l = l.replace('^', '.')
          l = l.replace('\n', ' ')
          l = l.replace('\t', ' ')
          l = l.replace('  ', ' ')
          l = l.replace('_',' ')
          text+=l[2:]
    return text

In [ ]:
def getTherSentences(lines):
    text = ""
    for i in range(3, len(lines)):
        if(lines[i-3][0] == 'E' and lines[i][0] == 'C'):
            text += lines[i-3]
            text += "| "
            text += lines[i]
            text += "$ "
    l = text
    l = l.replace('E1', 'E')
    l = l.replace('E2', 'E')
    l = l.replace('[CS]', '')
    l = l.replace('[FP]', '')
    l = l.replace('E', '')
    l = l.replace('C', '')
    l = l.replace('^', '.')
    l = l.replace('\n', ' ')
    l = l.replace('\t', ' ')
    l = l.replace('  ', ' ')
    l = l.replace('_',' ')
    return l

In [ ]:
def getTherSentences_(lines):
    text = ""
    for i in range(3, len(lines)):
        if(lines[i-3][0] == 'R' and lines[i][0] == 'C'):
            text += lines[i-3].split(':')[1]
            text += "| "
            text += lines[i].split(':')[1]
            text += "$ "
    l = text
    l = l.replace('[CS]', '')
    l = l.replace('[FP]', '')
    l = l.replace('^', '.')
    l = l.replace('\n', ' ')
    l = l.replace('\t', ' ')
    l = l.replace('  ', ' ')
    l = l.replace('_',' ')
    return l

def getChildSentences_(lines):
    lineCont = False
    text = ""
    for l in lines:
        if l[0]=='C':
          l = l.replace('[CS]', '')
          l = l.replace('[FP]', '')
          l = l.replace('^', '.')
          l = l.replace('\n', ' ')
          l = l.replace('\t', ' ')
          l = l.replace('  ', ' ')
          l = l.replace('_',' ')
          text+=l.split(':')[1]
    return text

In [ ]:
data = []
os.listdir("ASD_hinglish")
for fname in tqdm(range(18)):
    lines = open('ASD_hinglish/' + str(fname)+'.txt', 'r').readlines()
    data.append([str(fname)+'.txt', getChildSentences(lines), 0, getTherSentences(lines)])

In [ ]:
os.listdir("TD_hinglish")
files = ['539','540','541','542','544','545']
for fname in files:
    lines = open('TD_hinglish/' + str(fname)+'.txt', 'r').readlines()
    data.append([str(fname)+'.txt', getChildSentences_(lines), 1, getTherSentences_(lines)])

In [ ]:
df_hindi = pd.DataFrame(data, columns = ['ID', 'ChildText', 'Label', 'ChildTheraText'])

In [ ]:
df_hindi.to_csv('childesData_hinglish.csv')

## Reading Data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
features = pd.read_csv('features_pragmatic.csv', index_col = 0)

In [ ]:
features = features.to_numpy()

In [ ]:
label = features[:, 0]
features = features[:, 1:]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.1, random_state=0)

##Clan and Cohmetrix

In [ ]:
clan= pd.read_csv("CLAN_processed.csv")
cm=pd.read_csv("CohMetrix_processed.csv")

In [ ]:
cm.head()

,Unnamed: 0,TextID,DESPC,DESSC,DESWC,DESPL,DESPLd,DESSL,DESSLd,DESWLsy,...,WRDIMGc,WRDMEAc,WRDPOLc,WRDHYPn,WRDHYPv,WRDHYPnv,RDFRE,RDFKGL,RDL2,label
0,0,030400.txt,1,181,234,181,0,1.293,0.584,1.406,...,478.019012,444.247009,3.110,7.072,1.629,2.120,86.574997,1.505,42.106998,0.0
1,1,030610.txt,1,154,196,154,0,1.266,0.583,1.296,...,519.351990,444.016998,2.899,7.536,1.920,2.138,95.901001,0.199,34.491001,0.0
2,2,030806.txt,1,204,316,204,0,1.539,0.802,1.215,...,472.067993,476.445007,3.169,6.985,1.996,2.380,100.000000,0.000,35.362999,0.0
3,3,030900.txt,1,124,244,124,0,1.952,1.261,1.193,...,457.263000,435.282990,4.486,6.097,1.566,1.957,100.000000,0.000,46.977001,0.0
4,4,031009.txt,1,118,213,118,0,1.797,1.181,1.235,...,503.346008,467.026001,3.050,7.347,2.193,2.406,100.000000,0.000,24.642000,0.0


In [ ]:
def clan_cohMetrix(ID):
  ret=[]
  for i in range(len(clan)):
    if (clan["File"][i][:-4]==ID):
      ret.append(clan["density"][i])
  for i in range(len(cm)):
    if (cm["TextID"][i][:-4]==ID):
      ret.append(cm["PCREFz"][i])
      ret.append(cm["WRDAOAc"][i])
      ret.append(cm["WRDCNCc"][i])
  return ret